# Лекция 10

## Декораторы

>**Декоратор**— это функция, которая позволяет обернуть другую функцию для расширения её функциональности без непосредственного изменения её кода.

Раз мы знаем, как работают функции высших порядков, теперь мы можем понять как работают декораторы. Сначала посмотрим на пример декоратора:

In [ ]:
def decorator_function(func):
    def wrapper():
        print('Функция-обёртка!')
        print(f'Оборачиваемая функция: {func}')
        print('Выполняем обёрнутую функцию...')
        func()
        print('Выходим из обёртки')
    return wrapper

Здесь ```decorator_function()``` является функцией-декоратором. Как вы могли заметить, она является функцией высшего порядка, так как принимает функцию в качестве аргумента, а также возвращает функцию. 

Внутри ```decorator_function()``` мы определили другую функцию, обёртку, так сказать, которая обёртывает функцию-аргумент и затем изменяет её поведение. Декоратор возвращает эту обёртку. 

Теперь посмотрим на декоратор в действии:

In [ ]:
@decorator_function
def hello_world():
    print('Hello world!')
    
    
hello_world()


Функция-обёртка!
Оборачиваемая функция: <function hello_world at 0x7fd2cb512cb0>
Выполняем обёрнутую функцию...
Hello world!
Выходим из обёртки


Просто добавив ```@decorator_function``` перед определением функции ```hello_world()```, мы модифицировали её поведение. 

Однако, выражение с ```@``` является всего лишь синтаксическим сахаром для ```hello_world = decorator_function(hello_world)```.

Иными словами, выражение ```@decorator_function``` вызывает ```decorator_function()``` с ```hello_world``` в качестве аргумента и присваивает имени ```hello_world``` возвращаемую функцию.


Посмотрим на пример более полезного декоратора:

In [ ]:
import time
import requests


def benchmark(func):
    def wrapper():
        start = time.time()
        func()
        end = time.time()
        print(f'[*] Время выполнения: {end-start} секунд.')
    return wrapper

@benchmark
def fetch_webpage():
    webpage = requests.get('https://google.com')

fetch_webpage()

[*] Время выполнения: 1.5248000621795654 секунд.


Здесь мы создаём декоратор, замеряющий время выполнения функции. Далее мы используем его на функции, которая делает GET-запрос к главной странице Google. Чтобы измерить скорость, мы сначала сохраняем время перед выполнением обёрнутой функции, выполняем её, снова сохраняем текущее время и вычитаем из него начальное.

### Используем аргументы и возвращаем значения

В приведённых выше примерах декораторы ничего не принимали и не возвращали. Модифицируем наш декоратор для измерения времени выполнения:

In [ ]:
def benchmark(func):
    import time

    def wrapper(*args, **kwargs):
        start = time.time()
        return_value = func(*args, **kwargs)
        end = time.time()
        print(f'[*] Время выполнения: {end-start} секунд.')
        return return_value
    return wrapper

@benchmark
def fetch_webpage(url):
    import requests
    webpage = requests.get(url)
    return webpage.text

webpage = fetch_webpage('https://google.com')
print(webpage)

[*] Время выполнения: 1.41640305519104 секунд.
<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ru"><head><meta content="&#1055;&#1086;&#1080;&#1089;&#1082; &#1080;&#1085;&#1092;&#1086;&#1088;&#1084;&#1072;&#1094;&#1080;&#1080; &#1074; &#1080;&#1085;&#1090;&#1077;&#1088;&#1085;&#1077;&#1090;&#1077;: &#1074;&#1077;&#1073; &#1089;&#1090;&#1088;&#1072;&#1085;&#1080;&#1094;&#1099;, &#1082;&#1072;&#1088;&#1090;&#1080;&#1085;&#1082;&#1080;, &#1074;&#1080;&#1076;&#1077;&#1086; &#1080; &#1084;&#1085;&#1086;&#1075;&#1086;&#1077; &#1076;&#1088;&#1091;&#1075;&#1086;&#1077;." name="description"><meta content="noodp" name="robots"><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="Z4Hg55zUz5dA69Epw1mDCQ">(function(){window.google={kEI:'ypdyY5iwEfTJpgeH9ZyQBQ',kEXPI:'0,1302536,56873,6059,206,2414,2390,2316,383,246,5,5367,1123753

Как вы видите, аргументы декорируемой функции передаются функции-обёртке, после чего с ними можно делать что угодно. Можно изменять аргументы и затем передавать их декорируемой функции, а можно оставить их как есть или вовсе забыть про них и передать что-нибудь совсем другое. То же касается возвращаемого из декорируемой функции значения, с ним тоже можно делать что угодно.

### Декораторы с аргументами

Мы также можем создавать декораторы, которые принимают аргументы. Посмотрим на пример:

In [ ]:
import time
import requests


def benchmark(iters):
    def actual_decorator(func):
        def wrapper(*args, **kwargs):
            total = 0
            for i in range(iters):
                start = time.time()
                return_value = func(*args, **kwargs)
                end = time.time()
                total = total + (end-start)
            print(f'[*] Среднее время выполнения: {total/iters} секунд.')
            return return_value

        return wrapper
    
    return actual_decorator


@benchmark(iters=10)
def fetch_webpage(url):
    webpage = requests.get(url)
    return webpage.text

webpage = fetch_webpage('https://google.com')
print(webpage)

[*] Среднее время выполнения: 1.1625517368316651 секунд.
<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ru"><head><meta content="&#1055;&#1086;&#1080;&#1089;&#1082; &#1080;&#1085;&#1092;&#1086;&#1088;&#1084;&#1072;&#1094;&#1080;&#1080; &#1074; &#1080;&#1085;&#1090;&#1077;&#1088;&#1085;&#1077;&#1090;&#1077;: &#1074;&#1077;&#1073; &#1089;&#1090;&#1088;&#1072;&#1085;&#1080;&#1094;&#1099;, &#1082;&#1072;&#1088;&#1090;&#1080;&#1085;&#1082;&#1080;, &#1074;&#1080;&#1076;&#1077;&#1086; &#1080; &#1084;&#1085;&#1086;&#1075;&#1086;&#1077; &#1076;&#1088;&#1091;&#1075;&#1086;&#1077;." name="description"><meta content="noodp" name="robots"><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="jt8ns19AgahaBeQZHrjvTw">(function(){window.google={kEI:'P5hyY_OcHdOO4-EProySuAs',kEXPI:'0,1302536,56873,6058,207,4804,2316,383,246,5,5367,1

Здесь мы модифицировали наш старый декоратор таким образом, чтобы он выполнял декорируемую функцию ```iters``` раз, а затем выводил среднее время выполнения. Однако чтобы добиться этого, пришлось воспользоваться природой функций в Python.

Функция ```benchmark()``` на первый взгляд может показаться декоратором, но на самом деле таковым не является. Это обычная функция, которая принимает аргумент ```iters```, а затем возвращает декоратор. В свою очередь, он декорирует функцию ```fetch_webpage()```. Поэтому мы использовали не выражение @benchmark, а ```@benchmark(iters=10)``` — это означает, что тут вызывается функция ```benchmark()``` (функция со скобками после неё обозначает вызов функции), после чего она возвращает сам декоратор.

Да, это может быть действительно сложно уместить в голове, поэтому держите правило:

>**Декоратор принимает функцию в качестве аргумента и возвращает функцию.**

В нашем примере ```benchmark()``` не удовлетворяет этому условию, так как она не принимает функцию в качестве аргумента. В то время как функция ```actual_decorator()```, которая возвращается ```benchmark()```, является декоратором.

### Декораторы можно вкладывать друг в друга

In [11]:
def bread(func):
    def wrapper():
        print("</------\>")
        func()
        print("<\______/>")
    return wrapper
 
def ingredients(func):
    def wrapper():
        print("#помидоры#")
        func()
        print("~салат~")
    return wrapper

@bread
@ingredients
def sandwich(food="--ветчина--"):
    print(food)
    
sandwich()

</------\>
#помидоры#
--ветчина--
~салат~
<\______/>


Следует помнить о том, что порядок декорирования ВАЖЕН:

In [12]:
def bread(func):
    def wrapper():
        print("</------\>")
        func()
        print("<\______/>")
    return wrapper
 
def ingredients(func):
    def wrapper():
        print("#помидоры#")
        func()
        print("~салат~")
    return wrapper

@ingredients
@bread
def sandwich(food="--ветчина--"):
    print(food)
    
sandwich()

#помидоры#
</------\>
--ветчина--
<\______/>
~салат~


Как бы реализация бургера выглядела бы без синтаксического сахара декораторов:

In [13]:
def bread(func):
    def wrapper():
        print("</------\>")
        func()
        print("<\______/>")
    return wrapper
 
def ingredients(func):
    def wrapper():
        print("#помидоры#")
        func()
        print("~салат~")
    return wrapper


def sandwich(food="--ветчина--"):
    print(food)
    

sandwich = bread(ingredients(sandwich))
sandwich()

</------\>
#помидоры#
--ветчина--
~салат~
<\______/>


## Виртуальное окружение для Python

https://pythonist.ru/virtualnye-okruzheniya-python-i-instrumenty-dlya-upravleniya-imi/

https://docs.conda.io/en/latest/miniconda.html

## Основы создания телеграмм бота на python

допишу позже